In [1]:
!pip install opencv-python

In [19]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load model and class labels
model = load_model(r"C:\Users\Admin\AwakeVsDrowsiness\AwakeVsDrowsiness.h5")
class_labels = ['AWAKE', 'SLEEPY']

# Get all image paths
img_dir = r"C:\Users\Admin\AwakeVsDrowsiness\Data\Challenge"
img_files = [f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Predict all images
predictions = []
for file in img_files:
    path = os.path.join(img_dir, file)
    img = image.load_img(path, target_size=(150, 150))
    arr = image.img_to_array(img) / 255.0
    arr = np.expand_dims(arr, axis=0)
    pred = model.predict(arr, verbose=0)[0]
    predictions.append((file, pred))

# Sort by SLEEPY probability (descending)
predictions.sort(key=lambda x: x[1][1], reverse=True)

# Show top 3 "sleepiest" images
for file, pred in predictions[:3]:
    path = os.path.join(img_dir, file)
    img = cv2.imread(path)
    img = cv2.resize(img, (400, 400))

    awake, sleepy = pred[0] * 100, pred[1] * 100
    label = f"AWAKE: {awake:.1f}%, SLEEPY: {sleepy:.1f}%"
    color = (0, 255, 0) if awake >= sleepy else (0, 0, 255)

    cv2.putText(img, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
    cv2.imshow(file, img)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [21]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load model and labels
model = load_model(r"C:\Users\Admin\AwakeVsDrowsiness\AwakeVsDrowsiness.h5")
labels = ['AWAKE', 'SLEEPY']

# Load video
cap = cv2.VideoCapture(r"C:\Users\Admin\AwakeVsDrowsiness\Data\Challenge\Preview.mp4")

frame_interval = 30  # Predict every ~1 second
frame_count = 0
delay = 50  # ~20 FPS display speed

last_text = ""
last_color = (0, 255, 0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_interval == 0:
        resized = cv2.resize(frame, (150, 150)) / 255.0
        input_arr = np.expand_dims(resized, axis=0)
        pred = model.predict(input_arr, verbose=0)[0]
        awake, sleepy = pred[0] * 100, pred[1] * 100

        last_text = f"AWAKE: {awake:.1f}%, SLEEPY: {sleepy:.1f}%"
        last_color = (0, 255, 0) if awake >= sleepy else (0, 0, 255)

    # Annotate frame
    display = frame.copy()
    font, scale, thick = cv2.FONT_HERSHEY_SIMPLEX, 1.2, 2
    text_size, _ = cv2.getTextSize(last_text, font, scale, thick)
    w, h = text_size
    cv2.rectangle(display, (10, 10), (30 + w, 30 + h), last_color, -1)
    cv2.putText(display, last_text, (20, 40 + h // 2), font, scale, (255, 255, 255), thick)

    cv2.imshow("Driver Drowsiness Detection", display)
    if cv2.waitKey(delay) & 0xFF == ord('q'):
        break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()
